# Projet 6 : Analysez les ventes d'une librairie avec R ou Python

Un version retravaillée de ce notebook est également déployé sous Streamlit : https://armeldt-p6streamlit-home-x8ggr7.streamlit.app/

## Les données

In [96]:
import pandas as pd
import numpy as np
import matplotlib as mtp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import altair as alt
import scipy as sp
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
alt.data_transformers.disable_max_rows()

In [98]:
customers = pd.read_csv('Source_données/customers.csv')
products = pd.read_csv('Source_données/products.csv')
transactions = pd.read_csv('Source_données/transactions.csv')

In [ ]:
customers.head()

In [ ]:
products.head()

In [ ]:
transactions.head()

## Data préparation

### Table Client

Sur ce dataframe, on va venir vérifier le type des données, l'unicité de la clé primaire 'client_id' ainsi que la présence de donnée manquantes : 

In [ ]:
customers.dtypes

In [ ]:
customers['client_id'].unique().shape

In [ ]:
customers['client_id'].size

In [ ]:
customers.info

In [ ]:
customers.isna().sum(axis = 0)

La clé 'client_id' est bien unique (une ligne par client) et le dataframe ne comporte aucunes données vides, on peut donc le considérer comme exploitable pour nos analyses

### Table Produits

On va appliquer les mêmes vérifications sur ce dataframe Produit :

In [ ]:
products.dtypes

In [ ]:
products['id_prod'].unique().shape

In [ ]:
products['categ'].unique().shape

In [ ]:
products['id_prod'].size

In [ ]:
products.isna().sum(axis = 0)

La clé primaire de la table produit (id_prod) est également unique (une ligne par produit) et le dataframe ne présente aucunes lignes vides.

### Table transactions

Enfin on va également appliquer cette logique de vérification sur la table transaction :

In [ ]:
transactions.dtypes

In [ ]:
transactions.info

In [ ]:
transactions['client_id'].unique().shape

In [ ]:
transactions['id_prod'].unique().shape

On constate que la colonne "date" est renseignée en type 'object' on va donc la convertir en type 'datetime' pour exploitation future : 

In [ ]:
# En essayant de convertir les données au format datetime, une erreur apparait -> on constate que la présence d'entrées "test" nous empêche de convertir nos date vers le type datetime 

transactions[transactions['date'].str.contains('test')]

Après avoir isolé ces entrées test, on comprends que ce sont très certainement des essai mis en place par l'equipe de la librairie sur le site (en observant les id produit / id session / id client), on va donc les écarter pour pouvoir exploiter cette table 

In [117]:
# on va drop les lignes comprenant l'id produit 'T_0'
transactions.drop(transactions[transactions.id_prod == 'T_0'].index, inplace=True)

In [ ]:
# puis on vérifie que les valeurs test ont bien disparu
transactions[transactions['date'].str.contains('test')]

In [119]:
#on converti ensuite les valeurs date d'object vers datetime
transactions['date'] = pd.to_datetime(transactions['date'])

In [ ]:
# enfin on vérifie que la conversion a bien fonctionnée 
transactions.dtypes

In [ ]:
transactions.head()

In [ ]:
transactions.isna().sum(axis = 0)

Puisque la table répertorie des transactions, nous n'avons pas d'unicité a constater, cette dernière présente les bons types de données et aucuns doublons, elle est donc bonne pour être epxloitée dans le cadre de nos analyses.

### Table Globale

Afin de pouvoir exploiter pleinement les données que nous avons a disposition nous allons réaliser plusieurs fusions : 

Dans un premier temps on va enrichir le dataframe Transactions avec les informations de la table product (le prix et la catégorie) et de la table client (birth & sex).

On va appeler cette nouvelle table 'Globale'

In [ ]:
fusion1 = pd.merge (transactions, customers, on="client_id", how="left")
dfGlobal = pd.merge(fusion1, products, on="id_prod", how="left")
dfGlobal.head()

Puis on va vérifier l'intégrité des données comme pour nos tables source : 

In [ ]:
dfGlobal.info

In [ ]:
dfGlobal.dtypes
#afin de pouvoir l'exploiter plus facilement on va passer le code catégorie en tant qu'information catégorielle
dfGlobal['categ'] = dfGlobal['categ'].astype(str)
dfGlobal.dtypes

In [ ]:
dfGlobal.isna().sum(axis = 0)

On constate que les colonnes 'price' & 'categ' comportent 221 entrées nulles, on va fouiller afin de retrouver la raison de cette absence de valeurs

In [ ]:
testNA = dfGlobal[dfGlobal['price'].isna()]
testNA

In [ ]:
testNA['id_prod'].unique().shape

Il semblerait qu'un seul id_prod soit concerné par cette absence de prix, voyons voir si ce produit existe dans la table product originale

In [ ]:
products.loc[products['price'] == '0_2245',:]

Le produit n'est pas renseigné dans la table products, à défaut de pouvoir connaitre son prix, on va donc le retirer de notre table fusionnée pour le moment, tout prevenant la personne en gestion du e-commerce de cette absence :

In [130]:
dfGlobal.dropna(subset=['price'],inplace=True)

In [ ]:
dfGlobal.isna().sum(axis = 0)

### Enrichissement de la Table Globale

On va ensuite venir enrichir cette table globale avec des informations supplémentaires afin de pouvoir affiner nos analyses par la suite :

In [ ]:
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()

# Calculer l'âge des clients en soustrayant leur année de naissance de l'année actuelle
dfGlobal['age'] = date.year - dfGlobal['birth'].astype(int)

#ajouter la notion de tranche d'age dans le DF
bin_labels=labels=['18-24','25-34','35-44','45-60','61+']
dfGlobal["tranche_age"] = pd.cut(x=dfGlobal['age'], bins=[18,24,34,44,60,100],labels=bin_labels)

dfGlobal

Cette table enrichie comprends une ligne par transactions, il serait également intéressant de la grouper pour avoir ces informations par clients et par produits : 

Dans un premier temps on va créer une table avec la somme et le nombre de transactions de chaque clients en ajoutant des champs calculés supplémentaires permis par ce regroupement :
* la fréquence d'achat
* le panier moyen

In [ ]:
# grouper les achats par client et conserver la date de leur premier achat, celle de leur dernier achat, la somme des montants dépensés et le nombre de sessions realisées
dfGlobalcli = dfGlobal.groupby(['client_id','age','sex']).agg({'date': ['min', 'max',],'price':'sum','session_id':'count'}).reset_index()
dfGlobalcli.columns = ['_'.join(col) for col in dfGlobalcli.columns]
#on renomme les colonnes pour plus de lisibilité et pour eviter les accents & espaces
dfGlobalcli = dfGlobalcli.rename(columns={'client_id_':'client_id','age_':'age','sex_':'sex','date_min':'premier_achat','date_max':'dernier_achat', 'session_id_count' : 'nb_achats', 'price_sum':'CA'})
#on drop les clients n'ayant pas réalisé d'achats
dfGlobalcli = dfGlobalcli[dfGlobalcli['nb_achats'] >= 1]
# on calcule la durée entre le premier achat et le dernier achat en date afin de calculer une fréquence d'achat mensuelle moyenne (+ gestion des clients ayant acheté sur une seule date)
dfGlobalcli['periode_moyenne_jours'] = (dfGlobalcli['dernier_achat'] - dfGlobalcli['premier_achat']).dt.days
mask = dfGlobalcli['periode_moyenne_jours'] != 0
dfGlobalcli.loc[mask, "freq_achat_mensuelle"] = round((dfGlobalcli['nb_achats'] / dfGlobalcli['periode_moyenne_jours']) * 30,1)
dfGlobalcli.loc[~mask, "freq_achat_mensuelle"] = 0
# on ajoute la notion de panier moyen par client
dfGlobalcli["panier_moyen"] = round((dfGlobalcli['CA'] / dfGlobalcli['nb_achats']),1)
# et on ajoute les tranches d'age
bin_labels=labels=['18-24','25-34','35-44','45-60','61+']
dfGlobalcli["tranche_age"] = pd.cut(x=dfGlobalcli['age'], bins=[18,24,34,44,60,100],labels=bin_labels)
dfGlobalcli

Puis une seconde table avec la même logique mais avec les ta somme des transactions par produits :

In [ ]:
# On groupe le CA et le nombre de ventes par code produit :
dfGlobalProd = dfGlobal.groupby(['id_prod', 'categ']).agg({'price': ['sum', 'mean'], 'date': 'count'}).reset_index()
dfGlobalProd.columns = ['id_prod', 'categ', 'CA', 'prix_unitaire', 'nb_ventes']
dfGlobalProd

In [135]:
#export pour streamlit
dfGlobalProd.to_csv('Streamlit/Source_données/dfGlobalProd.csv', index=False)
dfGlobalcli.to_csv('Streamlit/Source_données/dfGlobalcli.csv', index=False)
dfGlobal.to_csv('Streamlit/Source_données/dfGlobal.csv', index=False)

On à désormais a notre disposition les 3 tables sources nettoyées : 
* Produits
* Clients
* Transactions

Et trois nouvelles tables :
* Une table 'globale' (la table transaction enrichie avec les données des tables Produits et Clients)
* Une table 'globalecli' qui regroupe notre table globale groupée par clients
* Une table 'globaleprod' qui regroupe notre table globale groupée par produits

## Analyse des indicateurs de vente

### 1. Indicateurs et graphiques autour du chiffre d'affaire

In [ ]:
#calcul du chiffre d'affaire
CA = dfGlobal['price'].sum()
print("le E-commerce a réalisé", round(CA,2),"€ de chiffre d'affaire de Mars 2021 à Février 2023")

#### Réalisation d'un histogramme sur l'évolution du CA dans le temps

In [ ]:
dfGlobal.head()

In [ ]:
#on va grouper notre dfGlobal par mois en aditionnant le CA réalisé par transactions pour avoir le CA global par mois dans un nouveau DF :
caMensuel = dfGlobal.drop(['categ','birth','age'], axis=1).groupby(pd.Grouper(key='date', freq='M')).sum('price')
caMensuel = caMensuel.reset_index()
caMensuel['date'] = pd.to_datetime(caMensuel['date'], format='%Y-%m')
caMensuel = caMensuel.rename(columns={"price": "CA"})
caMensuel.head()

In [ ]:
#puis on va le plotter dans un line chart pour analyser son évolution au cours des mois :
caMensuel_chart = alt.Chart(caMensuel).mark_area(line={'color':'blue'},color='blue').encode(
    x=alt.X('yearmonth(date):O', axis=alt.Axis(title='Mois')),
    y=alt.Y('CA', axis=alt.Axis(title="Chiffre d'affaires")),
    tooltip=['date', 'CA']
).properties(
    title="Chiffre d'affaires mensuel",
    width=1000
)
caMensuel_chart

On constate une forte baisse du CA en octobre, qui pourrait être due a une baisse naturelle de ventes suite a la rentrée scolaire qui représente un maronnier important dans le business de la librairie. 

Vérifions la véracité de cette hypothèse :

In [ ]:
# Sélection des lignes avec la date en octobre 2021
selected_df = dfGlobal.loc[dfGlobal['date'].between('2021-10-01', '2021-10-31'),:]
selected_df = selected_df.drop(['birth','age'], axis=1).groupby([pd.Grouper(key='date', freq='D'),'categ']).sum('price')
selected_df = selected_df.reset_index()
selected_df['date'] = pd.to_datetime(selected_df['date'], format='%Y-%m')
selected_df = selected_df.rename(columns={"price": "CA"})
selected_df
# Affichage dans un graphz
alt.Chart(selected_df).mark_bar().encode(
    x='date',
    y='sum(CA)',
    color='categ',
    tooltip=["CA","date"]
).properties(width=600)

Nous n'avons aucunes ventes sur la catégorie 1 du 2 au 27 octobre ce qui ressemble fortement a un souci de remontées des données de ventes depuis le e-commerce, il faudrait contacter le webmaster afin d'en savoir plus à ce sujet.

#### Décomposition en moyenne mobile pour évaluer la tendance globale

In [ ]:
# Calcul du chiffre d'affaires mensuel avec la moyenne mobile sur 3 mois
caMensuel_chart = caMensuel[['date', "CA"]]
caMensuel_chart['Moyenne mobile'] = caMensuel_chart['CA'].rolling(window=3).mean()

# Tracé du graphique en ligne avec la moyenne mobile personnalisée
chart = alt.Chart(caMensuel_chart).mark_line(point=True).encode(
    x=alt.X('yearmonth(date):O', axis=alt.Axis(title='Mois')),
    y=alt.Y("CA", axis=alt.Axis(title="Chiffre d'affaires")),
    tooltip=['date', "CA"]
).properties(
    title="Chiffre d'affaires mensuel avec moyenne mobile sur " + str(3) + " mois"
)

# Tracé de la moyenne mobile
rolling_avg_chart = alt.Chart(caMensuel_chart).mark_line(point=alt.MarkConfig(color='red'), color='red').encode(
    x='yearmonth(date):O',
    y='Moyenne mobile',
    tooltip=['date', 'Moyenne mobile']
)

chart = (chart + rolling_avg_chart).properties(height=300, width=1000)

chart

L'analyse de cette moyenne mobile a trois mois indique une tendance d'evolution de notre CA plutôt neutre, avec des valeurs de CA cumulé mensuel très proche de la moyenne des trois mois précédents (excepté pour mois d'octobre mais qui semble impacté par un manque de données). Il en va de même lorsqu'on passe la fenetre de moyenne mobile a 6 ou 12 mois.

#### Tops et Flops références

Les tops produits

In [ ]:
dfGlobalProd.sort_values(by='CA', ascending = False).head(10)

L'affichage du top10 des références ayant généré le plus de CA permet deja de constater plusieurs tendances :
* Aucun produit de la catégorie 0 n'est présent dans ce top
* 8 des 10 produits proviennent de la catégorie 2, catégorie dont le prix moyen semble le plus important
* Les deux produits issue de la catégorie 1 sont ceux qui ont fait l'objet du plus de ventes unitaires

Les flops

In [ ]:
dfGlobalProd.sort_values(by='CA', ascending = True).head(10)

Le top 10 des référnces ayant généré le moins de CA sur le site est quand à lui trusté par la catégorie 0 dont les produits semblent avoir un  prix moyen largement inférieur aux deux auitres catégories du site. 

### 2. Profiling Client

#### CA par clients via la courbe de Lorenz

Dans un premier temps on va créer un dataframe qui stock le CA par client

In [ ]:
# on va ensuite convertir notre colonne de CA par client en array numpy afin de faciliter le calcul du coefficient de gini et la création de la courbe de lorenz
caClientArr = dfGlobalcli['CA'].to_numpy()
# on vient egalement ordonner de manière croissante notre liste de valeurs
caClientArr = np.sort(caClientArr)
caClientArr

Pour ce faire on a besoin de calculer le coefficient de Gini : 

In [145]:
def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

In [ ]:
print("Coefficient de gini", round(gini(caClientArr),2))

Notre coefficient de Gini etant supérieur a 0.35 (seuil qui représente communément la frontière entre une répartition égalitaire et inégalitaire d'une série de valeurs), on peut deja déduire que le chiffre d'affaire de la libraire Lapage est répartie de manière inégalitaire entre ses clients. Voyons en détail comment ce dernier est répartie via la courbe de Lorenz

In [ ]:
def lorenz(caClientArr):
    # Division de la somme cumulative du CA par client par la somme totale du CA puis passage en %
    # ça permet egalement d'assurer que toutes les valeurs sont comprises entre 0 et 100
    scaled_prefix_sum = caClientArr.cumsum() / caClientArr.sum()*100
    # On ajoute la valeur 0 (car 0% des clients représentent 0% du chiffre d'affaire du magasin)
    return np.insert(scaled_prefix_sum, 0, 0)

# Affichage du coefficient de gini calculé plus haut
print("coefficient de Gini",round(gini(caClientArr),2))

y= lorenz(caClientArr)
#linspace -> création des points de la courbe de lorenz, dans l'interval 0,1, avec le nb de points extraits de notre fonction au dessus via le .size
x = np.linspace(0.0, 1.0, len(y))*100


plt.plot(x, y, label = "Courbe de Lorenz")
# Affichage de la droit d'égalité parfaite 
plt.plot([0,100], [0,100], label = "Droite de pente unitaire")
#ajout de titre et légende
plt.title("Répartition du chiffre d'affaire par clients")
plt.legend(loc = 'best', frameon = False)
plt.xlabel("Part cumulée des clients (en %)")
plt.ylabel("Part cumulée du chiffre d'affaire (en %)")
#affichage
plt.show()

La visualisation de la courbe de Lorenz nous permet de confirme que la répartition du chiffre d'affaire par clients est inégalitaire. A titre d'exemple, on peut constater que les 20% des clients les plus dépensiers génèrent a eux seuls quasiment 50% du CA de la boutique.

On peut également constater au sommet de la courbe de lorenz, qu'une poignée de client représente a eux seuls environ 8 a 10% du chiffre d'affaire de la boutique, il serait intéressant de creuser afin de comprendre qui sont ces clients.

In [ ]:
dfGlobalcli.sort_values(by='CA', ascending = False).head(10)

On va stocker dans un dataframe spécifique ces clients dont le CA (supérieur a 100k€) témoigne d'une consommation de livre plus proche de celle d'une entreprise que d'un particulier

In [149]:
clientsBtoB = ['c_1609','c_4958','c_6714','c_3454']

## Analyse de la clientèle

Afin de ne pas impacter nos différentes analyses de la clientèle avec bruit produit par les comportements très atypiques des clients BtoB identifiés, nous allons séparer le segment BtoC du segment BtoB. 

Afin de pouvoir apporter une analyse constructive pour le e-commerce de la librairie Lapage, nous allons uniquement nous concentrer sur les clients BtoC !

In [150]:
clientsBtoB = ['c_1609','c_4958','c_6714','c_3454']
dfGlobalcliBtoB = dfGlobalcli.loc[dfGlobalcli['client_id'].isin(clientsBtoB),:]
dfGlobalcliBtoC = dfGlobalcli.loc[~dfGlobalcli['client_id'].isin(clientsBtoB),:]
transactionsBtoC = dfGlobal[~dfGlobal['client_id'].isin(clientsBtoB)].reset_index(drop=True)

### 1. Le lien entre le genre d’un client et les catégories des livres achetés

On va dans un premier temps grouper notre dataframe global par sexe et catégorie de produit afin de pouvoir analyser une potentielle relation existante entre ces deux variables catégorielles :

In [ ]:
catxgenre = dfGlobal.groupby(['sex', 'categ']).sum('price')
catxgenre = catxgenre.reset_index()
catxgenre = catxgenre.loc[:, ['sex', 'categ', 'price']].rename(columns={'categ':'Catégories','price' : "CA"})
catxgenre['proportion en %'] = round(((catxgenre['CA'] / catxgenre.groupby('Catégories')['CA'].transform('sum')) * 100),2)
catxgenre.sort_values(by='Catégories')

On va ensuite plotter ce dataframe groupé afin de pouvoir l'analyser de manière plus visuelle :

In [ ]:
catxgenre_bar = alt.Chart(catxgenre).mark_bar(binSpacing=2).encode(
x=alt.X('Catégories:N', axis=alt.Axis(labelAngle=0)),
y='proportion en %',
color='sex:N',
tooltip=['proportion en %','Catégories']
).properties(title="Catégorie de livre achetée par genre"
).interactive()
catxgenre_bar

A première vue le genre du client semble avoir un très faible impact sur sa catégorie de prédilection, vérifions cela a l'aide d'un test statistique avec les deux hypothèses suivantes :

* H0 - Le sexe et la catégorie de livre achetée sont deux variables liées par une corrélation
* H1 - Le sexe et la catégorie de livre achetée ne présentent pas de corrélation

Puisque nous allons comparer deux variables qualitatives, on va se servir du test de Khi2 afin de determiner quelle hypothèse est vérifiée


Pour ce faire on va d'abord créer un tableau de contingence sur la base de notre Dataframe global et effectuer le test de khi2 

In [ ]:
from scipy.stats import chi2_contingency

# Calcul du tableau de contingence
contingence_table = pd.crosstab(dfGlobal['sex'], dfGlobal['categ'])

#réalisation du test de khi2 sur le tableau de contingence fraîchement créé : 
chi2, p_value, _, _ = chi2_contingency(contingence_table)

#affichage des resultats
print("Tableau de contingence :")
print(contingence_table)
print("\nRésultats du test du khi-deux :")
print("Statistique du khi-deux :", round(chi2,2))
print("p-valeur :", round(p_value,2)) 

On va compléter notre test statistique par le calcul du V de cramer pour définir si la corrélation est aussi faible qu'il y parait sur notre graphique et dans les premiers resultats du khi2 :

In [ ]:
def cramers_v(contingency_table):
    chi2, _, _, _ = sp.stats.chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    phi2 = chi2 / n
    r, k = contingency_table.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    v = (phi2corr / min((kcorr-1), (rcorr-1)))**0.5
    return v

# Calcul du tableau de contingence
contingency_table = pd.crosstab(dfGlobal['sex'], dfGlobal['categ'])

# Calcul du V de Cramer
v = cramers_v(contingency_table)

# Affichage du résultat
print("V de Cramer :", v)


Dans notre cas de figure, le resultat du test de Khi2 présentant une valeur de p inférieur a 0,05 signifie qu'il existe une corrélation statistiquement significative entre les variable de sexe et de catégories, et qu'on peut tirer l'enseignement suivant : 

-Les femmes sont plus suceptibles d'acheter des livres de la catégorie 1 alors que les hommes sont eux plus sucpetibles d'acheter des livres des catégories 0 et 2. 

Cependant dans notre cas, même si la correlation statistique est avérée, la méthode du V de cramer vient confirmer que la corrélation substantielle reste elle anecdotique avec un écart extremement faible entre la répartition par sexe sur les trois catégories, ce qui me semble nous empêcher de tirer une quelconque conclusion a appliquer d'un point de vue métier / activité. 

### 2. Le lien entre l’âge des clients et le montant total des achats, la fréquence d’achat, la taille du panier moyen et les catégories des livres achetés

#### Montant dépensé par age

Afin de permettre une analyse plus lisible, on va venir créer des tranche d'âges afin de représenter l'ensemble des catégories d'acheteurs de la librairie par des tranches de vies "communes" d'un point de vue marketing :


*   Les étudiants (18 - 24 ans) 

*   Les jeunes actifs (25 - 34 ans) 

*   Les actifs + parentalité (35 - 44 ans)

*   Les actifs  (45 - 60 ans) 

*   Les Seniors (61 ans et plus) 


On va ensuite créer un dataframe venant grouper le chiffre d'affaire globale du e-commerce par ces tranche d'âge recemment crées 

In [ ]:
montantParTrancheAge = dfGlobalcliBtoC.groupby(["tranche_age"]).agg({'CA':'sum', 'nb_achats':'sum',}).rename(columns={'price' : 'CA'}).reset_index().sort_values('CA', ascending=False)

On affiche ensuite ce dataframe sous forme de bar plot afin de faciliter sa lecture :

In [ ]:
montantParTrancheAge_chart = alt.Chart(montantParTrancheAge).mark_bar().encode(
x=alt.X("tranche_age", axis=alt.Axis(labelAngle=0)),
y='CA',
tooltip=['CA','tranche_age']).properties(
title="Chiffre d'affaire par tranche d\'age",
width=500)
montantParTrancheAge_chart 

Deux tranches semblent sortir du lot en matière de dépenses : les 35-44 et les 45-60 ans

En complément de cette analyse du chiffre d'affaire cumulé par tranche d'âge, on va afficher la répartition des clients par CA et par tranche d'âge :

In [ ]:
# dfGlobalcliBtoC.boxplot('CA', by='tranche_age')
alt.Chart(dfGlobalcliBtoC).mark_boxplot(size=20).encode(
alt.X("tranche_age", axis=alt.Axis(labelAngle=0)),
y='CA',
tooltip=['CA','tranche_age']).properties(
title="Chiffre d'affaire par tranche d\'age",
width=400)

A l'oeil nu, il semblerait que le CA dépensé soit corrélé à la tranche d'age du client, on va donc émettre deux hyptohèses : 

   - H0 : Il n'y a pas de corrélation linéaire entre âge et montant total des achats
   - H1 : Il y a  corrélation linéaire entre âge et montant total des achats 

Afin de définir quel test employer pour démontrer ou non cette corrélation, on va devoir analyser un peu plus en détail notre variable quantitative, le Chiffre d'affaire, afin de connaitre notamment sa distribution.

In [ ]:
statistic, p_value = sp.stats.shapiro(dfGlobalcliBtoC['CA'])
alpha = 0.05

if p_value > alpha:
    print("La variable quantitative suit une distribution normale.")
else:
    print("La variable quantitative ne suit pas une distribution normale.")

Le test de Shapiro effectué sur la répartition du chiffre d'affaire réfute l'hypothèse ou ce dernier est ditribué normalement, vérifions cela a l'aide d'un affichage en graphique :

In [ ]:
# Création des données
X = dfGlobalcliBtoC['CA']
x_min = 0
x_max = np.max(X)
mean = np.mean(X)
std = np.std(X)
x = np.linspace(x_min, x_max, 100)
y = sp.stats.norm.pdf(x, mean, std)

# Création du dataframe
data = pd.DataFrame({'x': x, 'y': y})

# Graphique
line = alt.Chart(data).mark_line(color='blue').encode(
    x='x',
    y='y'
).properties(
    title='CA : comparaison avec une distribution normale',
    width=400,
    height=300
)

distrib = alt.Chart(dfGlobalcliBtoC).transform_density(
    'CA',
    as_=['CA', 'density'],
).mark_area(color='indianred',
            fillOpacity=0.5,
            stroke='indianred',
            strokeWidth=2).encode(
    x="CA:Q",
    y='density:Q',
)

distribCA_chart = distrib + line
chart.configure_axis(
    labelFontSize=10,
    titleFontSize=10
).configure_legend(
    title=None
).configure_title(
    fontSize=10
)
distribCA_chart

On constate visuellement que la courbe de répartition (en rouge) de notre variable de chiffre d'affaire ne suit pas la courbe de distribution normale affichée en bleu ce qui confirme le résultat du test de Shapiro : 

- Notre variable de CA ne suit pas une distribution normale

Puisque notre variable quantitative (le CA) ne suit pas une distribution normale, nous allons devoir vérifier la corrélation avec un test non paramétrique, on va utiliser celui de  Kruskal-Wallis qui est l'équivalent de l'ANOVA mais qui viens comparer les médianes entre les groupes d'age plutôt que le moyennes.

In [ ]:
tranche_age_1 = dfGlobalcliBtoC[dfGlobalcliBtoC['tranche_age'] == '45-60']['CA']
tranche_age_2 = dfGlobalcliBtoC[dfGlobalcliBtoC['tranche_age'] == '35-44']['CA']
tranche_age_3 = dfGlobalcliBtoC[dfGlobalcliBtoC['tranche_age'] == '25-34']['CA']
tranche_age_4 = dfGlobalcliBtoC[dfGlobalcliBtoC['tranche_age'] == '18-24']['CA']
tranche_age_5 = dfGlobalcliBtoC[dfGlobalcliBtoC['tranche_age'] == '61+']['CA']

# Effectuer le test de Kruskal-Wallis
statistic, p_value = sp.stats.kruskal(tranche_age_1, tranche_age_2, tranche_age_3, tranche_age_4, tranche_age_5)

# Afficher les résultats
print("Statistique de test :", round(statistic,2))
print("Valeur p :", round(p_value,2))

La statisque résultant du test de Kruskal-Wallis (404,2) suggère qu'il existe une différence significative entre nos différents groupes.

la valeur extrêmement faible de notre p confirme cette observation : Les chiffres d'affaires par tranche d'âge sont statistiquement différents les uns des autres.

Afin de consolider le resultat de ce premier test, on va également réaliser un ANOVA, en testant au préalable la variance de nos différentes tranche d'âge :

In [ ]:
# Effectuer le test de Levene
levene_stat, levene_pvalue = sp.stats.levene(*[dfGlobalcliBtoC[dfGlobalcliBtoC['tranche_age'] == cat]['CA'] for cat in dfGlobalcliBtoC['tranche_age'].unique()])

# Afficher les résultats
print("Résultats du test de Levene:")
print("Statistique de test :", round(levene_stat,2))
print("Valeur de p :", round(levene_pvalue,2))

Les variance ne sont pas égales, on va donc appliquer un test Welch-ANOVA

In [ ]:

# Effectuer l'ANOVA de Welch
model = ols('CA ~ tranche_age', data=dfGlobalcliBtoC).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Afficher les résultats
print("Résultats de l'ANOVA de Welch:")
print(anova_table)

Comme pour le test de Kruskal-Wallis, la p value est inférieur a < 0,5 on peut rejeter l'hypothèe d'indépendance et affirmer que notre variable CA à une différence de distribution significative pour nos différentes tranches d'âge et ainsi établir un lien significatif entre nos variables.

Il peut être intéressant pour la librairie de prendre cette enseignement et d'adapter de fait leur offre et/ou leurs efforts en terme de marketing afin de séduire cette cible qui semble la plus dépensière

#### Fréquence d'achats

Afin d'analyse un potentiel rapport entre fréquence d'achat et âge des clients de la librairie, on va afficher ces variables sur un nuage de point :

In [ ]:
# Création du nuage de points avec Altair
frequenceAchatAge_scatterplot = alt.Chart(dfGlobalcliBtoC).mark_circle(size=60).encode(
    x='age',
    y='nb_achats',
    color='freq_achat_mensuelle',
    tooltip=['age', 'nb_achats', 'freq_achat_mensuelle']
).properties(
    width=600,
    height=400
)
frequenceAchatAge_scatterplot

Visuellement, il semble émerger une corrélation entre l'age et la fréquence d'achat, avec les plus hautes fréquences (en bleu foncé) majoritairement situées dans la tranche 35-50 ans, vérifion si cette corrélation est statistiquement démontrable par des tests :

Puisque nous souhaitons analyser la corrélation entre deux variable quantitatives (âge et frequence d'achat) nous allons devoir vérifier dans un premier temps si leur distribution est normale ou non via un test de shapiro : 

In [ ]:
statistic, p_value = sp.stats.shapiro(dfGlobalcliBtoC['freq_achat_mensuelle'])
alpha = 0.05

if p_value > alpha:
    print(round(p_value,2), "La variable fréquence d'achat moyenne mensuelle suit une distribution normale.")
else:
    print(round(p_value,2), "La variable fréquence d'achat moyenne mensuelle ne suit pas une distribution normale.")

statistic, p_value = sp.stats.shapiro(dfGlobalcliBtoC["age"])
alpha = 0.05

if p_value > alpha:
    print(round(p_value,2), "La variable age suit une distribution normale.")
else:
    print(round(p_value,2), "La variable age ne suit pas une distribution normale.")

Selon les resultats du test de Shapiro, nos deux variables ne sont pas distribuées normalement, vérifions cela a l'aide de visualisations :

In [ ]:
###### DISTRIB AGE ######

# Création des données pour la distribution d'âge
X_age = dfGlobalcliBtoC['age']
x_min_age = 0
x_max_age = np.max(X_age)
mean_age = np.mean(X_age)
std_age = np.std(X_age)
x_age = np.linspace(x_min_age, x_max_age, 100)
y_age = sp.stats.norm.pdf(x_age, mean_age, std_age)

# Création du dataframe pour la distribution d'âge
data_age = pd.DataFrame({'x': x_age, 'y': y_age})

# Graphique pour la distribution d'âge
line_age = alt.Chart(data_age).mark_line(color='blue').encode(
    x='x',
    y='y'
).properties(
    title='Age : comparaison avec une distribution normale',
    width=400,
    height=300
)

distribAge = alt.Chart(dfGlobalcliBtoC).transform_density(
    'age',
    as_=['age', 'density']
).mark_area(color='indianred',
            fillOpacity=0.5,
            stroke='indianred',
            strokeWidth=2).encode(
    x="age:Q",
    y='density:Q'
)

distribAge_chart = distribAge + line_age

###### DISTRIB FREQ ACHAT ######

# Création des données pour la distribution de fréquence d'achat
X_freq = dfGlobalcliBtoC['freq_achat_mensuelle']
x_min_freq = 0
x_max_freq = np.max(X_freq)
mean_freq = np.mean(X_freq)
std_freq = np.std(X_freq)
x_freq = np.linspace(x_min_freq, x_max_freq, 100)
y_freq = sp.stats.norm.pdf(x_freq, mean_freq, std_freq)

# Création du dataframe pour la distribution de fréquence d'achat
data_freq = pd.DataFrame({'x': x_freq, 'y': y_freq})

# Graphique pour la distribution de fréquence d'achat
line_freq = alt.Chart(data_freq).mark_line(color='blue').encode(
    x='x',
    y='y'
).properties(
    title='Freq_achat_mensuelle : comparaison avec une distribution normale',
    width=400,
    height=300
)

distribFreq = alt.Chart(dfGlobalcliBtoC).transform_density(
    'freq_achat_mensuelle',
    as_=['freq_achat_mensuelle', 'density']
).mark_area(color='indianred',
            fillOpacity=0.5,
            stroke='indianred',
            strokeWidth=2).encode(
    x="freq_achat_mensuelle:Q",
    y='density:Q'
)

distribFreq_chart = distribFreq + line_freq

combined_chart = alt.hconcat(distribAge_chart, distribFreq_chart)

combined_chart.configure_axis(
    labelFontSize=10,
    titleFontSize=10
).configure_legend(
    title=None
).configure_title(
    fontSize=10
)


Ces deux graphiques confirment bien l'absence de normalité dans la distribution des deux variables que sont l'âge et la fréquence d'achat.

Puisque nos deux variables ne suivent pas une distribution normale on va utiliser le test non paramétrique de Spearman afin de confirmer ou infirmer la corrélation entre la fréquence d'achat et l'âge des clients de la librairie


On va dans un premier temps formuler les deux hypothèses H0 et H1 : 

   - H0 : Il n'y a pas de corrélation linéaire entre âge et la fréquence d'achats par session.
   - H1 : Il y a corrélation linéaire entre âge et la fréquence d'achats par session.

In [ ]:
correlation, p_value = sp.stats.spearmanr(dfGlobalcliBtoC["freq_achat_mensuelle"], dfGlobalcliBtoC["age"])

# Afficher le résultat du test de corrélation de Spearman
print("Coefficient de corrélation de Spearman :", round(correlation,2))
print("p-value :", round(p_value,2))

le resultat du test viens valider notre hypothèse qu'une corrélation linéaire existe bien entre âge et fréquence d'achats par mois, même si cette dernière reste très faible avec un coefficient Spearman de 0.12.

#### Le panier moyen



Afin d'analyser le lien entre l'âge et le panier moyen, on va également plotter ces donées dans un nuage de point :

In [ ]:
# Création du nuage de points avec Altair
panierMoyenAge_scatterplot = alt.Chart(dfGlobalcli).mark_circle(size=60).encode(
    x='age',
    y='panier_moyen',
    color='panier_moyen',
    tooltip=['age', 'panier_moyen']
).properties(
    width=400,
    height=300
)
panierMoyenAge_scatterplot

Ce dernier semble souligner une corrélation assez nette entre panier moyen et age du client, vérifions si cette corrélation est statistiquement vérifiable à l'aide d'un test statsitique : 

Dans un premier temps, on va tester la distribution de la variable panier_moyen :

In [ ]:
# Création des données pour la distribution de fréquence d'achat
X_freq = dfGlobalcliBtoC['panier_moyen']
x_min_freq = 0
x_max_freq = np.max(X_freq)
mean_freq = np.mean(X_freq)
std_freq = np.std(X_freq)
x_freq = np.linspace(x_min_freq, x_max_freq, 100)
y_freq = sp.stats.norm.pdf(x_freq, mean_freq, std_freq)

# Création du dataframe pour la distribution de fréquence d'achat
data_freq = pd.DataFrame({'x': x_freq, 'y': y_freq})

# Graphique pour la distribution de fréquence d'achat
line_freq = alt.Chart(data_freq).mark_line(color='blue').encode(
    x='x',
    y='y'
).properties(
    title='Panier_moyen : comparaison avec une distribution normale',
    width=400,
    height=300
)

distribFreq = alt.Chart(dfGlobalcliBtoC).transform_density(
    'panier_moyen',
    as_=['panier_moyen', 'density']
).mark_area(color='indianred',
            fillOpacity=0.5,
            stroke='indianred',
            strokeWidth=2).encode(
    x="panier_moyen:Q",
    y='density:Q'
)

distribFreq_chart = distribFreq + line_freq
distribFreq_chart 

In [ ]:
statistic, p_value = sp.stats.shapiro(dfGlobalcliBtoC['panier_moyen'])
alpha = 0.05

if p_value > alpha:
    print(p_value, "La variable panier moyen suit une distribution normale.")
else:
    print(p_value, "La variable panier moyen ne suit pas une distribution normale.")


Nos deux variables ont une distribution non normale, on va donc devoir appliquer un test non paramétrique, celui de Pearson

In [ ]:
correlation, p_value = sp.stats.spearmanr(dfGlobalcliBtoC["panier_moyen"], dfGlobalcliBtoC["age"])

# Afficher le résultat du test de corrélation de Spearman
print("Coefficient de corrélation de Spearman :", round(correlation,2))
print("p-value :", round(p_value,2))

Le coéfficient de corrélation de Spearman viens confirmer ce que nous avions pu constater a l'oeil nu sur le nuage de point : une corrélation négative significative existe bien entre le panier moyen et l'age des clients.

Les clients les plus jeunes (18-32 ans) ont tendances a acheter des paniers plus remplis que leurs ainés (33 ans et plus)

#### Catégories des livres achetés

Calcul du CA dépensé par tranchage d'age et par catégorie

In [ ]:
#repartition des achats par categ par tranche d'age 
catxTrancheAge = transactionsBtoC.groupby(['tranche_age', 'categ']).sum('price')
catxTrancheAge = catxTrancheAge.reset_index()
catxTrancheAge = catxTrancheAge.loc[:, ['tranche_age', 'categ', 'price']].rename(columns={'categ':'Catégories','price' : "CA"})
catxTrancheAge['proportion en %'] = round(((catxTrancheAge['CA'] / catxTrancheAge.groupby('Catégories')['CA'].transform('sum')) * 100),2)
catxTrancheAge.sort_values(by='Catégories')

On va venir visualiser ce dataframe a l'aide d'un bar plot pour plus de lisibilité :

In [ ]:
catxTrancheAge_bar = alt.Chart(catxTrancheAge).mark_bar(binSpacing=4).encode(
x=alt.X('Catégories:N', axis=alt.Axis(labelAngle=0)),
y='proportion en %',
color='tranche_age:N',
tooltip=['proportion en %','Catégories']
).properties(width=100,height=400)
catxTrancheAge_bar

Sur ce graph, il semblerait que la tranche d'age ait une corrélation avec la catégorie de livre achetée : 
- la catégorie 2 est majoritairement plébicité par les deux tranches les plus jeunes (18-24 et 25-34)
- La catégorie 0 est a l'inverse majoritairement achetée par les tranches d'age supérieures (35-44 et 45-60)
- la catégorie 1 est quand a elle mieux divisée entre chaque tranche d'age mais egalement la catégorie la plus achetée par les séniors(61+)

On va vérifier cette dernière a l'aide d'un test de Khi2 sur ces deux variables qualitatives que sont les tranche d'âge et les catégories d'achat :

In [ ]:
contingency_table = pd.crosstab(transactionsBtoC['tranche_age'], transactionsBtoC['categ'])

# Effectuez le test du Chi carré
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Affichez les résultats
print('Test du Chi carré :')
print('Valeur Chi carré :', chi2)
print('p-value :', p_value)

In [ ]:
# Calcul du tableau de contingence
contingency_table = pd.crosstab(transactionsBtoC['tranche_age'], transactionsBtoC['categ'])

# Calcul du V de Cramer
v = cramers_v(contingency_table)

# Affichage du résultat
print("V de Cramer :", v)

le test de chi2 nous permet de confirmer qu'il existe une corrélation significative entre la tranche d'âge et la catégorie de livres achetés par les clients de la libraire (hors BtoB). Cette observation est cependant pondérée par le test du V de Cramer qui indique par son coéfficient de 0,37 que l'association entre les deux variables étudiées (Tranche d'age et catégorie) reste modérée.

En complément de cette première analyse, on va regarder également observer la répartition du volume d'achats par âge et par catégorie. On va visualiser a l'aide d'un nuage de point :

In [ ]:
# Création du nuage de points avec Altair
Categ_age_scatterplot = alt.Chart(transactionsBtoC).mark_circle(size=60).encode(
    x='categ',
    y='age',
    color='count(age)',
    tooltip=['count(age)','categ', 'age']
).properties(
    width=400,
    height=300
)
Categ_age_scatterplot

La même corrélation semble se dégager qu'avec les tranches d'âge : 

* Les clients de -30 ans semblent être ceux qui achètent le plus sur la catégorie 2 
* Les clients entre 30 et 50 ans semblent acheter en majorité sur la catégorie 0 
* La catégorie 1 semble répartie de manière plus égalitaire entre les âges

Afin de vérifier une potentielle corrélation entre ces deux variables et en connaissance du fait de la non nomralité de la distrubtion de l'âge, on va utiliser le test non paramétrique de Kruskal-wallis :

In [ ]:
kruskal_stat_cat_age, kruskal_pvalue_cat_age = sp.stats.kruskal(*[transactionsBtoC[transactionsBtoC['categ'] == cat]['age'] for cat in transactionsBtoC['categ'].unique()])

print("Test de Kruskal-Wallis pour 'cat_age':")
print("Statistique de test de Kruskal-Wallis :", kruskal_stat_cat_age)
print("Valeur de p :", kruskal_pvalue_cat_age)

Le resultat de notre test de Kruskal-Wallis sur le lien entre la catégorie est l'âge démontre qu'une différence significative entre nos groupes existe belle et bien, avec une valeur de p inférieur a 0,05.

## Conclusion

Suite aux différentes analyses réalisées sur la clientèle du site, nous avons pu établir que des corélations statistiquements prouvées liaient plusieurs indicateurs de performance clé du site (CA, fréquence d'achat, panier moyen, catégorie d'achat) avec l'âge des clients. 

De ces analyses émergent 3 profils de clients ayant des habitudes de consommation bien distinctes et dans lesquels on va pouvoir classer les utilisateurs du site pour mieux les adresser :


**Les étudiants et jeunes actifs** (dont la tranche d'âge est située entre 18 et 32 ans)

Ce profil comprends les clients ayant le plus fort panier moyen mais également les clients achetant le moins fréquemment.

On constate sur ce graphique que les pics de consommation de cette cible sont en Juillet et Aout, en amont de la rentrée scolaire.

Concernant cette cible deux actions me semblent intéressantes :

* Adapter l'offre dédiée à cette cible (scolaire) a l'approche de la rentrée de septembre pour faire gonfler le chiffre d'affaire
* Essayer de fidéliser cette clientèle à l'occasion des maronniers que sont la rentrée et la fin d'année en l'incitant à revenir sur le e-commerce pour ses loisirs sur d'autres periodes via de l'action commerciale (bon de réduction, frais de livraison offert etc...)

**Les actifs** (dont la tranche d'âge est située entre 33 et 51 ans)

Ce profil comprends les clients ayant généré le plus de CA au sein de la boutique en ligne, ce notamment grâce a une très forte fréquence d'achat malgré un panier moyen plus faible que les jeunes actifs & étudiants.

Cette cible semble acheter souvent des produits des catégories 1 et 2 mais en petite quantitée à chaque fois. Pourquoi ne pas essayer de profiter de cette forte fréquence d'achat pour pousser ces clients à ajouter plus de produits à leur panier et ainsi générer plus de CA à l'aide d'actions commerciales ou d'une stratégie de prix revisitée par exemple.

**Les seniors** (dont la tranche d'age et de 50 ans et plus)

Cette cible regroupe les clients ayant le plus faible CA, avec une fréquence d'achat et un panier moyen dans la moyenne basse

Elle semble être la moins adressée des trois par le e-commerce, que ce soit dans l'offre (les achats sont répartis entres les catégories 0 et 1, pas de catégorie de prédilection contrairement aux deux autres cibles) ou dans les actions de communication (faible CA, faible fréquence d'achat...)

Il serait intéressant de comparer ces données avec leur équivalent sur les magasins physiques, cette cible etant probablement plus habituée au commerce qu'au e-commerce et en parallèle développer une offre leur permettant de compléter leur achats en magasin avec des achats sur le e-commerce. 